In [134]:
import pandas as pd
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import pymongo
import json
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup
from googlesearch import search 

In [46]:
df = pd.read_csv('Resources/Starbucks_Menu.csv')
df.head()

,Beverage_Id,Beverage,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine
0,1,Brewed Coffee,Beverage,5,0.1,10,0,0,1.0,330
1,2,CaffA Latte,Beverage,190,7.0,170,0,17,12.0,150
2,3,CaffA Mocha (Without Whipped Cream),Beverage,260,8.0,140,2,34,13.0,175
3,4,Vanilla Latte (Or Other Flavoured Latte),Beverage,250,6.0,150,0,35,12.0,150
4,5,CaffA Americano,Beverage,15,0.0,15,0,0,1.0,225


In [47]:
df_lkp = df[['Beverage_Id', 'Beverage']].copy()
bev_lkp = df_lkp.to_numpy()
df_lkp.head(10)

,Beverage_Id,Beverage
0,1,Brewed Coffee
1,2,CaffA Latte
2,3,CaffA Mocha (Without Whipped Cream)
3,4,Vanilla Latte (Or Other Flavoured Latte)
4,5,CaffA Americano
5,6,Cappuccino
6,7,Espresso
7,8,Skinny Latte (Any Flavour)
8,9,Caramel Macchiato
9,10,White Chocolate Mocha (Without Whipped Cream)


In [48]:
df_new = df.drop(['Beverage'],axis =1)
df_new.head()

,Beverage_Id,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine
0,1,Beverage,5,0.1,10,0,0,1.0,330
1,2,Beverage,190,7.0,170,0,17,12.0,150
2,3,Beverage,260,8.0,140,2,34,13.0,175
3,4,Beverage,250,6.0,150,0,35,12.0,150
4,5,Beverage,15,0.0,15,0,0,1.0,225


In [49]:
labelEncoder = LabelEncoder()
labelEncoder.fit(df_new['Type'])
df_new['Type'] = labelEncoder.transform(df_new['Type'])
df_new.head()

,Beverage_Id,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine
0,1,0,5,0.1,10,0,0,1.0,330
1,2,0,190,7.0,170,0,17,12.0,150
2,3,0,260,8.0,140,2,34,13.0,175
3,4,0,250,6.0,150,0,35,12.0,150
4,5,0,15,0.0,15,0,0,1.0,225


In [50]:
df_features = df_new[['Type','Calories','Fat','Carbohydrate','Fibre','Sugar', 'Protein' ,'Caffeine']].copy()
df_features.head()

,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine
0,0,5,0.1,10,0,0,1.0,330
1,0,190,7.0,170,0,17,12.0,150
2,0,260,8.0,140,2,34,13.0,175
3,0,250,6.0,150,0,35,12.0,150
4,0,15,0.0,15,0,0,1.0,225


In [51]:
df_label = df_new[['Beverage_Id']].copy()
label = df_label.to_numpy()

In [52]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_features)
X_scaled

array([[0.        , 0.00769231, 0.0027027 , ..., 0.        , 0.02941176,
        1.        ],
       [0.        , 0.29230769, 0.18918919, ..., 0.26153846, 0.35294118,
        0.45454545],
       [0.        , 0.4       , 0.21621622, ..., 0.52307692, 0.38235294,
        0.53030303],
       ...,
       [1.        , 0.52307692, 0.35135135, ..., 0.53846154, 0.52941176,
        0.        ],
       [1.        , 0.13846154, 0.21621622, ..., 0.        , 0.02941176,
        0.        ],
       [1.        , 0.13846154, 0.        , ..., 0.29230769, 0.02941176,
        0.        ]])

In [53]:
kmeans = KMeans(n_clusters=5, max_iter=300, algorithm = 'auto')
kmeans.fit(X_scaled)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [54]:
filename = 'SB_Menu_ML.h5'
pickle.dump(kmeans, open(filename, 'wb'))

In [55]:
loaded_model = pickle.load(open(filename, 'rb'))

In [56]:
predict_me = np.array(X_scaled[100].astype(float))
predict_me = predict_me.reshape(-1, len(predict_me))
prediction = loaded_model.predict(predict_me)
prediction[0]

0

In [88]:
for i in range(len(df_features)):
    predict_me = np.array(X_scaled[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = loaded_model.predict(predict_me)
    df.loc[df['Beverage_Id'] == label[i][0], 'Cluster'] = prediction[0] 
    
df.head()

,Beverage_Id,Beverage,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine,Cluster
0,1,Brewed Coffee,Beverage,5,0.1,10,0,0,1.0,330,2.0
1,2,CaffA Latte,Beverage,190,7.0,170,0,17,12.0,150,3.0
2,3,CaffA Mocha (Without Whipped Cream),Beverage,260,8.0,140,2,34,13.0,175,3.0
3,4,Vanilla Latte (Or Other Flavoured Latte),Beverage,250,6.0,150,0,35,12.0,150,3.0
4,5,CaffA Americano,Beverage,15,0.0,15,0,0,1.0,225,2.0


In [109]:
df.loc[df['Cluster'] == 0, 'Label'] = 'High Protien Low Sugar Meal'
df.loc[df['Cluster'] == 1, 'Label'] = 'Feel good snack'
df.loc[df['Cluster'] == 2, 'Label'] = 'Light Beverage'
df.loc[df['Cluster'] == 3, 'Label'] = 'Beverage'
df.loc[df['Cluster'] == 4, 'Label'] = 'Healthy Snack'

In [110]:
df

,Beverage_Id,Beverage,Type,Calories,Fat,Carbohydrate,Fibre,Sugar,Protein,Caffeine,Cluster,Label
0,1,Brewed Coffee,Beverage,5,0.1,10,0,0,1.0,330,2.0,Light Beverage
1,2,CaffA Latte,Beverage,190,7.0,170,0,17,12.0,150,3.0,Beverage
2,3,CaffA Mocha (Without Whipped Cream),Beverage,260,8.0,140,2,34,13.0,175,3.0,Beverage
3,4,Vanilla Latte (Or Other Flavoured Latte),Beverage,250,6.0,150,0,35,12.0,150,3.0,Beverage
4,5,CaffA Americano,Beverage,15,0.0,15,0,0,1.0,225,2.0,Light Beverage
5,6,Cappuccino,Beverage,120,4.0,100,0,10,8.0,150,2.0,Light Beverage
6,7,Espresso,Beverage,10,0.0,1,0,0,1.0,150,2.0,Light Beverage
7,8,Skinny Latte (Any Flavour),Beverage,120,0.3,160,0,16,12.0,150,3.0,Beverage
8,9,Caramel Macchiato,Beverage,240,7.0,150,0,32,10.0,150,3.0,Beverage
9,10,White Chocolate Mocha (Without Whipped Cream),Beverage,400,11.0,250,0,58,15.0,150,3.0,Beverage


In [58]:
df.to_csv('Resources/Starbucks_Menu_Clustered.csv')

In [59]:
df_0 = df.loc[df['Cluster'] == 0] 
df_1 = df.loc[df['Cluster'] == 1] 
df_2 = df.loc[df['Cluster'] == 2] 
df_3 = df.loc[df['Cluster'] == 3] 
df_4 = df.loc[df['Cluster'] == 4] 

In [103]:
print("Calories - Min, Max, Mean" ) , df_0['Calories'].min(), df_0['Calories'].max(), df_0['Calories'].mean()

Calories - Min, Max, Mean


(None, 310, 650, 476.6666666666667)

In [61]:
print("Carbohydrate - Min, Max, Mean" ) ,df_0['Carbohydrate'].min() , df_0['Carbohydrate'].max(), df_0['Carbohydrate'].mean()

Carbohydrate - Min, Max, Mean


(None, 9, 80, 44.638888888888886)

In [62]:
print("Fibre - Min, Max, Mean" ) ,df_0['Fibre'].min() , df_0['Fibre'].max(), df_0['Fibre'].mean()

Fibre - Min, Max, Mean


(None, 0, 21, 4.666666666666667)

In [63]:
print("Sugar - Min, Max, Mean" ) ,df_0['Sugar'].min() , df_0['Sugar'].max(), df_0['Sugar'].mean()

Sugar - Min, Max, Mean


(None, 1, 30, 9.166666666666666)

In [64]:
print("Caffeine - Min, Max, Mean" ) ,df_0['Caffeine'].min() , df_0['Caffeine'].max(), df_0['Caffeine'].mean()

Caffeine - Min, Max, Mean


(None, 0, 0, 0.0)

In [65]:
print("Calories - Min, Max, Mean" ) , df_1['Calories'].min(), df_1['Calories'].max(), df_1['Calories'].mean()


Calories - Min, Max, Mean


(None, 300, 490, 389.14285714285717)

In [66]:
print("Carbohydrate - Min, Max, Mean" ) ,df_1['Carbohydrate'].min() , df_1['Carbohydrate'].max(), df_1['Carbohydrate'].mean()

Carbohydrate - Min, Max, Mean


(None, 36, 68, 50.114285714285714)

In [67]:
print("Fibre - Min, Max, Mean" ) ,df_1['Fibre'].min() , df_1['Fibre'].max(), df_1['Fibre'].mean()

Fibre - Min, Max, Mean


(None, 0, 3, 1.5714285714285714)

In [68]:
print("Sugar - Min, Max, Mean" ) ,df_1['Sugar'].min() , df_1['Sugar'].max(), df_1['Sugar'].mean()


Sugar - Min, Max, Mean


(None, 7, 43, 28.0)

In [69]:
print("Caffeine - Min, Max, Mean" ) ,df_1['Caffeine'].min() , df_1['Caffeine'].max(), df_1['Caffeine'].mean()

Caffeine - Min, Max, Mean


(None, 0, 0, 0.0)

In [70]:
print("Calories - Min, Max, Mean" ) , df_2['Calories'].min(), df_2['Calories'].max(), df_2['Calories'].mean()

Calories - Min, Max, Mean


(None, 0, 120, 55.0)

In [71]:
print("Carbohydrate - Min, Max, Mean" ) ,df_2['Carbohydrate'].min() , df_2['Carbohydrate'].max(), df_2['Carbohydrate'].mean()

Carbohydrate - Min, Max, Mean


(None, 0, 100, 20.75)

In [72]:
print("Fibre - Min, Max, Mean" ) ,df_2['Fibre'].min() , df_2['Fibre'].max(), df_2['Fibre'].mean()

Fibre - Min, Max, Mean


(None, 0, 0, 0.0)

In [73]:
print("Sugar - Min, Max, Mean" ) ,df_2['Sugar'].min() , df_2['Sugar'].max(), df_2['Sugar'].mean()

Sugar - Min, Max, Mean


(None, 0, 24, 9.5)

In [74]:
print("Caffeine - Min, Max, Mean" ) ,df_2['Caffeine'].min() , df_2['Caffeine'].max(), df_2['Caffeine'].mean()

Caffeine - Min, Max, Mean


(None, 50, 330, 155.625)

In [75]:
print("Calories - Min, Max, Mean" ) , df_3['Calories'].min(), df_3['Calories'].max(), df_3['Calories'].mean()

Calories - Min, Max, Mean


(None, 120, 400, 248.2608695652174)

In [76]:
print("Carbohydrate - Min, Max, Mean" ) ,df_3['Carbohydrate'].min() , df_3['Carbohydrate'].max(), df_3['Carbohydrate'].mean()

Carbohydrate - Min, Max, Mean


(None, 20, 250, 171.7391304347826)

In [77]:
print("Fibre - Min, Max, Mean" ) ,df_3['Fibre'].min() , df_3['Fibre'].max(), df_3['Fibre'].mean()

Fibre - Min, Max, Mean


(None, 0, 7, 1.0)

In [78]:
print("Sugar - Min, Max, Mean" ) ,df_3['Sugar'].min() , df_3['Sugar'].max(), df_3['Sugar'].mean()

Sugar - Min, Max, Mean


(None, 16, 65, 41.69565217391305)

In [79]:
print("Caffeine - Min, Max, Mean" ) ,df_3['Caffeine'].min() , df_3['Caffeine'].max(), df_3['Caffeine'].mean()

Caffeine - Min, Max, Mean


(None, 0, 175, 82.3913043478261)

In [80]:
print("Calories - Min, Max, Mean" ) , df_4['Calories'].min(), df_4['Calories'].max(), df_4['Calories'].mean()

Calories - Min, Max, Mean


(None, 90, 380, 226.66666666666666)

In [81]:
print("Carbohydrate - Min, Max, Mean" ) ,df_4['Carbohydrate'].min() , df_4['Carbohydrate'].max(), df_4['Carbohydrate'].mean()


Carbohydrate - Min, Max, Mean


(None, 5, 70, 31.595238095238095)

In [82]:
print("Fibre - Min, Max, Mean" ) ,df_4['Fibre'].min() , df_4['Fibre'].max(), df_4['Fibre'].mean()

Fibre - Min, Max, Mean


(None, 0, 8, 2.357142857142857)

In [83]:
print("Sugar - Min, Max, Mean" ) ,df_4['Sugar'].min() , df_4['Sugar'].max(), df_4['Sugar'].mean()

Sugar - Min, Max, Mean


(None, 0, 29, 9.571428571428571)

In [84]:
print("Caffeine - Min, Max, Mean" ) ,df_4['Caffeine'].min() , df_4['Caffeine'].max(), df_4['Caffeine'].mean()

Caffeine - Min, Max, Mean


(None, 0, 0, 0.0)

In [195]:
#i = 0 

for index , row in df.iterrows():
    #i = i + 1    
    bev = row['Beverage']    
    query = "Starbucks" + bev    
    
    #if i < 10:
    for url in search(query, tld="co.in", num=1, start = 0, stop=1, pause=3): 
        #print(url)  
        df.loc[df['Beverage'] == bev, 'item_url'] = url
        try:
            executable_path = {'executable_path': 'chromedriver.exe'}
            browser = Browser('chrome', **executable_path, headless=True)
            browser.visit(url)
            html = browser.html
            sb_menu = BeautifulSoup(html, 'html.parser')
            sb_menu_div = sb_menu.find('div', attrs={"class": "region size1of3"})
            pic_url = "https:" + sb_menu_div.find('p').find('img')['src']
            df.loc[df['Beverage'] == bev, 'image_url'] = pic_url
        except:
            print("Error " + bev + ' ' + url )
            
    

Error Brewed Coffee https://www.starbucks.com/menu/drinks/brewed-coffee
Error Espresso https://www.starbucks.com/menu/drinks/espresso
Error Caramel Macchiato https://www.starbucks.com/coffee/espresso/caramel-macchiato-beverages
Error TazoA Tea https://www.usatoday.com/story/money/2017/11/02/starbucks-sells-tazo-tea-brand-univeler-focuses-teavana-instead/826529001/
Error TazoA Full-Leaf Tea Latte https://www.republicoftea.com/mate-latte-full-leaf/p/v00689/
Error Shaken Iced TazoA Tea (With Classic Syrup) https://delishably.com/dining-out/Starbucks-Drink-Guide-Iced-Teas
Error Coffee https://www.starbucks.com/
Error Coffee https://www.starbucks.com/
Error Cranberry Orange Scone https://www.laurafuentes.com/copycat-starbucks-cranberry-orange-scone-recipe/
Error Octopus Cookie https://www.reddit.com/r/ExpectationVsReality/comments/6e3tru/starbucks_octopus_cookie/
Error Pirate Cake Pop https://www.youtube.com/watch?v=0fX7OM6N_B4
Error Italian-Style Ham & Spicy Salami https://www.brandeating.

In [196]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.sb_menu_db
collection = db.menu_cluster
collection.drop()

records = json.loads(df.T.to_json()).values()
collection.insert_many(records)